# Project Solution

You have to work on the [Kiva](https://drive.google.com/file/d/1-tJtnIbo1Rt-F1XfoWGVkmBXiI-ciuRx/view) dataset. Some information on the datasets are on the [Kaggle](https://www.kaggle.com/gaborfodor/additional-kiva-snapshot) web page.

## Basic tasks

All groups and individual must do the following:

`1.` Normalize the `loan_lenders` table. In the normalized table, each row must have one `loan_id` and one `lender`.

Firstly,necesery libraries are imported:

In [1]:
import pandas as pd
import numpy as np

After that, csv file is read and data frame with unnormalized data is created:

In [2]:
loans_lenders_un = pd.read_csv('../../Datasets/kiva/loans_lenders.csv')
#loans_lenders_un = loans_lenders_un.head() # Coment this in real situation, uncomment during development
loans_lenders_un

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."
...,...,...
1387427,678999,"michael43411218, carol5987, gooddogg1, chris41..."
1387428,1207353,"rjhoward1986, jeffrey6870, trolltech4460, elys..."
1387429,1206220,"vicky7746, gooddogg1, fairspirit, craig9729960..."
1387430,1206425,"rich6705, sergiiy9766, angela7509, barbara5610..."


Data frame with structured data will be created with help of intermediate object - list of pairs (loan_id, lender) packed in the dictionary:

In [3]:
lis = []
for index, row in loans_lenders_un.iterrows(): 
    ls = row['lenders'].split(',')
    for l in ls:
        lis.append({ 'loan_id' : row['loan_id'], 'lender': l.strip() })
loans_lender = pd.DataFrame(lis) 
loans_lender 


,loan_id,lender
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


`2.` For each loan, add a column `duration` corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

Firstly, data frame should be loaded and structure of the data frame `loans` should be determined:

In [4]:
import pandas as pd
import numpy as np
loans = pd.read_csv('../../Datasets/kiva/loans.csv')
loans.columns

Index(['loan_id', 'loan_name', 'original_language', 'description',
       'description_translated', 'funded_amount', 'loan_amount', 'status',
       'activity_name', 'sector_name', 'loan_use', 'country_code',
       'country_name', 'town_name', 'currency_policy',
       'currency_exchange_coverage_rate', 'currency', 'partner_id',
       'posted_time', 'planned_expiration_time', 'disburse_time',
       'raised_time', 'lender_term', 'num_lenders_total',
       'num_journal_entries', 'num_bulk_entries', 'tags', 'borrower_genders',
       'borrower_pictured', 'repayment_interval', 'distribution_model'],
      dtype='object')

After thar, several values from the begining of the data frame should be displayed:

In [5]:
loans_head = loans.head()
#loans = loans.head() # Coment this in real situation, uncomment during development
loans_head

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


We are now mainly interested in columns 'planned_expiration_time', 'disburse_time':

In [6]:
loans_sel = loans[ ['loan_id', 'loan_name','planned_expiration_time', 'disburse_time'] ]
loans_sel

,loan_id,loan_name,planned_expiration_time,disburse_time
0,657307,Aivy,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000
1,657259,Idalia Marizza,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000
2,658010,Aasia,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000
3,659347,Gulmira,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000
4,656933,Ricky\t,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000
...,...,...,...,...
1419602,988180,NaN,2016-01-02 01:00:03.000 +0000,2015-11-23 08:00:00.000 +0000
1419603,988213,Perlita,2016-01-02 16:40:07.000 +0000,2015-11-24 08:00:00.000 +0000
1419604,989109,Okyeso Nyame Group,2016-01-03 22:20:04.000 +0000,2015-11-13 08:00:00.000 +0000
1419605,989143,Exequila,2016-01-05 08:50:02.000 +0000,2015-11-03 08:00:00.000 +0000


In [7]:
from datetime import datetime
loans['duration'] = None
for index, row in loans.iterrows(): 
    s2 = row['planned_expiration_time']
    s1 = row['disburse_time']
    if( pd.notna(s1) and pd.notna(s2) and s1 != '' and s2 != ''):
        d2 = datetime.strptime(s2, "%Y-%m-%d %H:%M:%S.%f %z")
        d1 = datetime.strptime(s1, "%Y-%m-%d %H:%M:%S.%f %z")
        loans.loc[index,'duration'] = (d2 - d1).days 
loans_sel = loans[ ['loan_id', 'loan_name','planned_expiration_time', 'disburse_time', 'duration'] ]
loans_sel

,loan_id,loan_name,planned_expiration_time,disburse_time,duration
0,657307,Aivy,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,53
1,657259,Idalia Marizza,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,96
2,658010,Aasia,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,37
3,659347,Gulmira,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,34
4,656933,Ricky\t,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,57
...,...,...,...,...,...
1419602,988180,NaN,2016-01-02 01:00:03.000 +0000,2015-11-23 08:00:00.000 +0000,39
1419603,988213,Perlita,2016-01-02 16:40:07.000 +0000,2015-11-24 08:00:00.000 +0000,39
1419604,989109,Okyeso Nyame Group,2016-01-03 22:20:04.000 +0000,2015-11-13 08:00:00.000 +0000,51
1419605,989143,Exequila,2016-01-05 08:50:02.000 +0000,2015-11-03 08:00:00.000 +0000,63


`3.` Find the lenders that have funded at least twice.

Those are lenders that are duplicated in structured loan_leneders dataframe.


- One possible solution is based on calculation/counting using dicitionary object:

In [8]:
stat={}
for index, row in loans_lender.iterrows(): 
    lender = row['lender']
    stat[lender] = 1 + stat.get(lender, 0)
lenders_funded_2_more = []
for k in stat.keys():
    if( stat[k] >= 2):
        lenders_funded_2_more.append((k,stat[k]))
lenders_funded_2_more

[('muc888', 914),
 ('sam4326', 680),
 ('camaran3922', 48),
 ('rebecca3499', 14995),
 ('karlheinz4543', 1047),
 ('jerrydb', 283),
 ('paula8951', 110),
 ('gmct', 128159),
 ('amra9383', 10),
 ('r3922', 9675),
 ('brian9451', 270),
 ('shree8053', 10036),
 ('alan5513', 8822),
 ('oisin3389', 3044),
 ('helle8622', 142),
 ('bo3186', 247),
 ('ric8947', 80),
 ('daniel98469874', 16119),
 ('nick9464', 19),
 ('deborah12671549', 680),
 ('matthew9831', 106),
 ('john6330', 1567),
 ('john9479', 339),
 ('mattiaslaven', 4735),
 ('jonathan2867', 52),
 ('jason3883', 994),
 ('highgrovechurch', 28742),
 ('maria3124', 25),
 ('dino5102', 1425),
 ('jonathan7946', 676),
 ('ann8187', 367),
 ('bryan2669', 201),
 ('john88459657', 111),
 ('eddyphil', 2397),
 ('don9212', 24578),
 ('carolineandcolin9686', 308),
 ('bent8782', 209),
 ('raph8817', 1287),
 ('danielle2350', 263),
 ('nora3555', 3),
 ('williammanashi', 33),
 ('barbara5610', 48925),
 ('danhostetler', 150),
 ('daniel1104', 1965),
 ('amirali5409', 34661),
 ('oce

In [9]:
loans_lender_2_more = loans_lender[loans_lender.duplicated(['lender'])]
lenders_funded_2_more = list(set(loans_lender_2_more['lender']))
lenders_funded_2_more

['crobin1313',
 'salvador6678',
 'domenico2152',
 'dana8274',
 'paul8821',
 'erica4587',
 'patricia4185',
 'staffan9141',
 'kristie8554',
 'anna2573',
 'queena7840',
 'luc6184',
 'emily1490',
 'adrienne6434',
 'sandra6009',
 'kiran7697',
 'denise2795',
 'carolina9548',
 'elaina3062',
 'brian7172',
 'jen2021',
 'rolfz',
 'akiko5445',
 'rebecca7763',
 'alexa6662',
 'craig1040',
 'sarah58888534',
 'diane3480',
 'neil6090',
 'claudia5101',
 'sophie3700',
 'mayajk',
 'maddiemii',
 'franziska2715',
 'dea4057',
 'bruno1001',
 'gary2135',
 'yeo',
 'johanna8821',
 'jim79973404',
 'james58372523',
 'shaun8919',
 'michael96315668',
 'noah7862',
 'elisabeth4705',
 'jana9106',
 'soumya8889',
 'martha92445468',
 'fred9624',
 'fredrik4493',
 'laura7000',
 'nira1920',
 'elyse6334',
 'emily9169',
 'alejandra8376',
 'chris5067',
 'michele9148',
 'abby19618027',
 'davidandbeverly5324',
 'ryan41641240',
 'keith4205',
 'melissa3097',
 'renata2808',
 'elizabeth5140',
 'nancy46041774',
 'fidel4069',
 'zach87

`4.` For each country, compute how many loans have involved that country as borrowers.

- One solution is based on calculation/counting using dicitionary object:

In [10]:
stat={}
for index, row in loans.iterrows(): 
    cn = row['country_name']
    cc = row['country_code']
    stat[(cc,cn)] = 1 + stat.get((cc,cn), 0)
stat

{('PH', 'Philippines'): 285336,
 ('HN', 'Honduras'): 15130,
 ('PK', 'Pakistan'): 45120,
 ('KG', 'Kyrgyzstan'): 11243,
 ('SV', 'El Salvador'): 64037,
 ('BI', 'Burundi'): 1727,
 ('ML', 'Mali'): 13352,
 ('MN', 'Mongolia'): 9659,
 ('PE', 'Peru'): 86000,
 ('GE', 'Georgia'): 6483,
 ('AM', 'Armenia'): 13952,
 ('GH', 'Ghana'): 22121,
 ('TZ', 'Tanzania'): 16110,
 ('KH', 'Cambodia'): 79701,
 ('TG', 'Togo'): 17106,
 ('GT', 'Guatemala'): 12864,
 ('LB', 'Lebanon'): 20083,
 ('PY', 'Paraguay'): 24787,
 ('NI', 'Nicaragua'): 42519,
 ('KE', 'Kenya'): 143699,
 ('UG', 'Uganda'): 45882,
 ('RW', 'Rwanda'): 16774,
 ('MZ', 'Mozambique'): 7086,
 ('AZ', 'Azerbaijan'): 10172,
 ('IL', 'Israel'): 464,
 ('TJ', 'Tajikistan'): 43942,
 ('BO', 'Bolivia'): 25250,
 ('MX', 'Mexico'): 19708,
 ('ID', 'Indonesia'): 10699,
 ('NG', 'Nigeria'): 20015,
 ('EC', 'Ecuador'): 33471,
 ('MG', 'Madagascar'): 4872,
 ('VN', 'Vietnam'): 21839,
 ('CO', 'Colombia'): 33675,
 ('JO', 'Jordan'): 10161,
 ('YE', 'Yemen'): 4206,
 ('IN', 'India'): 

- Another solution is based on panda grouping:

In [11]:
loans_count = loans.groupby(['country_code','country_name'])['loan_id'].count()
loans_count

country_code  country_name          
AF            Afghanistan                2337
AL            Albania                    3075
AM            Armenia                   13952
AZ            Azerbaijan                10172
BA            Bosnia and Herzegovina      608
                                        ...  
XK            Kosovo                     2178
YE            Yemen                      4206
ZA            South Africa                633
ZM            Zambia                     1277
ZW            Zimbabwe                   5513
Name: loan_id, Length: 95, dtype: int64

`5.` For each country, compute the overall amount of money borrowed.


- One solution is based on calculation/sum using dicitionary object:

In [12]:
stat={}
for index, row in loans.iterrows(): 
    cn = row['country_name']
    cc = row['country_code']
    stat[(cc,cn)] = stat.get((cc,cn), 0) + row['loan_amount']
stat

{('PH', 'Philippines'): 97984600.0,
 ('HN', 'Honduras'): 11989325.0,
 ('PK', 'Pakistan'): 24995850.0,
 ('KG', 'Kyrgyzstan'): 14726900.0,
 ('SV', 'El Salvador'): 41691550.0,
 ('BI', 'Burundi'): 5233450.0,
 ('ML', 'Mali'): 15504675.0,
 ('MN', 'Mongolia'): 15348375.0,
 ('PE', 'Peru'): 79437775.0,
 ('GE', 'Georgia'): 9570425.0,
 ('AM', 'Armenia'): 22950475.0,
 ('GH', 'Ghana'): 15262725.0,
 ('TZ', 'Tanzania'): 18720300.0,
 ('KH', 'Cambodia'): 51613525.0,
 ('TG', 'Togo'): 10022825.0,
 ('GT', 'Guatemala'): 18718050.0,
 ('LB', 'Lebanon'): 27397150.0,
 ('PY', 'Paraguay'): 53964700.0,
 ('NI', 'Nicaragua'): 30153225.0,
 ('KE', 'Kenya'): 66735975.0,
 ('UG', 'Uganda'): 38386500.0,
 ('RW', 'Rwanda'): 29157325.0,
 ('MZ', 'Mozambique'): 4181100.0,
 ('AZ', 'Azerbaijan'): 14784625.0,
 ('IL', 'Israel'): 1732725.0,
 ('TJ', 'Tajikistan'): 39622125.0,
 ('BO', 'Bolivia'): 44226725.0,
 ('MX', 'Mexico'): 25106300.0,
 ('ID', 'Indonesia'): 8104600.0,
 ('NG', 'Nigeria'): 7751725.0,
 ('EC', 'Ecuador'): 35519000.0,

- Another solution is based on panda grouping:

In [13]:
loans_sum = loans.groupby(['country_code','country_name'])['loan_amount'].sum()
loans_sum

country_code  country_name          
AF            Afghanistan                1967950.0
AL            Albania                    4307350.0
AM            Armenia                   22950475.0
AZ            Azerbaijan                14784625.0
BA            Bosnia and Herzegovina      477250.0
                                           ...    
XK            Kosovo                     3083025.0
YE            Yemen                      3444000.0
ZA            South Africa               1006525.0
ZM            Zambia                     1978975.0
ZW            Zimbabwe                   5851875.0
Name: loan_amount, Length: 95, dtype: float64

`6.` Like the previous point, but expressed as a percentage of the overall amount lent.


- One solution is based on calculation using dicitionary object:

In [14]:
stat={}
total = 0
for index, row in loans.iterrows(): 
    cn = row['country_name']
    cc = row['country_code']
    stat[(cc,cn)] = stat.get((cc,cn), 0) + row['loan_amount']
    total += row['loan_amount']
for k in stat.keys():
    stat[k] = stat[k]/total*100
stat

{('PH', 'Philippines'): 8.293677540060738,
 ('HN', 'Honduras'): 1.0148084032897895,
 ('PK', 'Pakistan'): 2.1157153240379327,
 ('KG', 'Kyrgyzstan'): 1.2465240432141427,
 ('SV', 'El Salvador'): 3.528883843433756,
 ('BI', 'Burundi'): 0.4429731480460284,
 ('ML', 'Mali'): 1.3123569909296076,
 ('MN', 'Mongolia'): 1.2991273426021002,
 ('PE', 'Peru'): 6.723824869927503,
 ('GE', 'Georgia'): 0.8100662642020867,
 ('AM', 'Armenia'): 1.9425893358877362,
 ('GH', 'Ghana'): 1.2918776984610187,
 ('TZ', 'Tanzania'): 1.5845360562088229,
 ('KH', 'Cambodia'): 4.368706236039779,
 ('TG', 'Togo'): 0.8483586052344885,
 ('GT', 'Guatemala'): 1.5843456102156248,
 ('LB', 'Lebanon'): 2.3189677522455066,
 ('PY', 'Paraguay'): 4.567715950732214,
 ('NI', 'Nicaragua'): 2.552249281447268,
 ('KE', 'Kenya'): 5.648710684858181,
 ('UG', 'Uganda'): 3.249135607958205,
 ('RW', 'Rwanda'): 2.4679536527245247,
 ('MZ', 'Mozambique'): 0.35389944096059944,
 ('AZ', 'Azerbaijan'): 1.2514100409729743,
 ('IL', 'Israel'): 0.14666245936199

- Another solution is based on panda functions:

In [15]:
loans_total = loans['loan_amount'].sum()
loans_percent = loans.groupby(['country_code','country_name'])['loan_amount'].sum()/loans_total*100
loans_percent

country_code  country_name          
AF            Afghanistan               0.166573
AL            Albania                   0.364586
AM            Armenia                   1.942589
AZ            Azerbaijan                1.251410
BA            Bosnia and Herzegovina    0.040396
                                          ...   
XK            Kosovo                    0.260955
YE            Yemen                     0.291509
ZA            South Africa              0.085195
ZM            Zambia                    0.167506
ZW            Zimbabwe                  0.495318
Name: loan_amount, Length: 95, dtype: float64

`7.` Like the three previous points, but split for each year (with respect to `disburse time`).


`8.` For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.


`9.` For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.


`10.` Which country has the highest ratio between the difference computed at the previous point and the population?


`11.` Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?


`12.` For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.